# How to solve equation 4

In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [2]:
# Sample x data from Gaussian distribution
np.random.seed(0)
n = 100

x = np.random.randn(n)

In [3]:
def generate_matrix_A(x: np.ndarray) -> list[np.ndarray, np.ndarray, np.ndarray]:
    n = x.shape[0]
    A_1 = np.zeros((n, n))
    A_2 = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            A_1[i, j] = np.maximum(0, x[i] - x[j])
            A_2[i, j] = np.maximum(0, x[j] - x[i])

    bar_A_1 = (np.eye(n) - np.ones((n, n)) / n) @ A_1
    bar_A_2 = (np.eye(n) - np.ones((n, n)) / n) @ A_2

    A = np.concatenate([bar_A_1, bar_A_1, bar_A_2, bar_A_2], axis=1)

    return A, A_1, A_2

A, A_1, A_2 = generate_matrix_A(x)

In [4]:
def generate_vector_b(x: np.ndarray) -> np.ndarray:
    n = x.shape[0]

    C_1 = np.zeros((n, n))
    C_2 = np.zeros((n, n))
    C_3 = np.zeros((n, n))
    C_4 = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            C_1[i, j] = np.where(x[i] - x[j] >= 0, 1, 0)
            C_2[i, j] = np.where(x[i] - x[j] > 0, 1, 0)
            C_3[i, j] = np.where(-x[i] + x[j] >= 0, 1, 0)
            C_4[i, j] = np.where(-x[i] + x[j] > 0, 1, 0)

    b = np.concatenate([np.ones(n).T @ C_1, np.ones(n).T @ C_2, -np.ones(n).T @ C_3, -np.ones(n).T @ C_4])
    return b

b = generate_vector_b(x)

## Solve the convex problem in Theorem 3.1

$min_y 0.5* ||Ay||^2_2 + b^Ty + \beta ||y||_1$

In [5]:
y = cp.Variable(4 * n)
beta = 1

score_matching = 0.5*cp.norm2(A@y)**2 + b.T@y + beta*cp.norm1(y)
objective = cp.Minimize(score_matching)

# Solve the optimization problem
prob = cp.Problem(objective)
result = prob.solve(verbose=True)
y_star = y.value

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Dec 11 12:59:58 PM: Your problem has 400 variables, 0 constraints, and 0 parameters.
(CVXPY) Dec 11 12:59:58 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 12:59:58 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 12:59:58 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 12:59:58 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 12:59:58 PM: Compiling problem (target solver=CLARABEL).

In [7]:
# Reconstruct weights
def reconstruct_weights(y: np.ndarray, x: np.ndarray, A_1: np.ndarray, A_2: np.ndarray, epsilon: np.float32 = 1e-6) -> list[np.ndarray, np.ndarray, np.ndarray, np.float32]:
    n = x.shape[0]

    w_star = np.zeros(4*n)
    for j in range(4*n):
        if j < 2*n:
            w_star[j] = np.sqrt(np.abs(y[j]))
        else:
            w_star[j] = -np.sqrt(np.abs(y[j]))

    alpha_star = np.sqrt(np.abs(y))
    b_star = np.zeros(4*n)
    for i in range(4*n):
        if i < n:
            b_star[i] = -np.sqrt(np.abs(y[i])) * x[i]
        elif i < 2*n:
            b_star[i] = -np.sqrt(np.abs(y[i])) * (x[i-n] + epsilon)
        elif i < 3*n:
            b_star[i] = np.sqrt(np.abs(y[i])) * x[i-2*n]
        else:
            b_star[i] = np.sqrt(np.abs(y[i])) * (x[i-3*n] - epsilon)

    b_0_star = -1/n * np.ones(n).T @ (np.concatenate([A_1, A_1, A_2, A_2], axis=1) @ y)
    return w_star, alpha_star, b_star, b_0_star

w_star, alpha_star, b_star, b_0_star = reconstruct_weights(y_star, x, A_1, A_2)
print(w_star.shape, alpha_star.shape, b_star.shape, b_0_star)

(400,) (400,) (400,) 1315.2539879825122


In [ ]:
# construct neural network
def NN(x: np.ndarray) -> np.ndarray:
    n = x.shape[0]

    h = alpha_star.T @ np.maximum(0, w_star.T @ x + b_star) + b_0_star
    return h